# Lexical Density & Sentence Length Analysis

This notebook performs a comparative analysis of Leo Tolstoy's *Anna Karenina* and *War and Peace*.

## Metrics Calculated:
1.  **Lexical Density:** The percentage of content words (Nouns, Verbs, Adjectives, Adverbs) vs. total tokens.
2.  **Sentence Length Distribution:** Statistical analysis of how long the sentences are in each book.

**Formula:** $Lexical Density = \frac{Total Content Words}{Total Words} \times 100$

In [ ]:
import os
import nltk
import matplotlib.pyplot as plt
import numpy as np
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tag import pos_tag

# --- Setup & Download NLTK Data ---
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt', quiet=True)
    nltk.download('averaged_perceptron_tagger', quiet=True)
    nltk.download('averaged_perceptron_tagger_eng', quiet=True)
    nltk.download('punkt_tab', quiet=True)

print("Libraries loaded and NLTK data ready.")

In [ ]:
def calculate_metrics(text):
    """
    Calculates two metrics:
    1. Lexical Density (percentage)
    2. List of sentence lengths (for box plot)
    """
    # --- 1. Lexical Density ---
    # Filter for alphabetic words only
    tokens = [word.lower() for word in word_tokenize(text) if word.isalpha()]
    total_words = len(tokens)
    
    if total_words == 0: return 0, []
    
    tagged_tokens = pos_tag(tokens)
    # Tags: N=Noun, V=Verb, J=Adjective, R=Adverb
    content_word_tags = ('N', 'V', 'J', 'R')
    content_words = [word for word, tag in tagged_tokens if tag.startswith(content_word_tags)]
    
    density = (len(content_words) / total_words) * 100
    
    # --- 2. Sentence Lengths ---
    sentences = sent_tokenize(text)
    # Count words in each sentence (filtering out punctuation)
    sentence_lengths = [len([w for w in word_tokenize(s) if w.isalpha()]) for s in sentences]
    
    return density, sentence_lengths

In [ ]:
# --- Analysis Logic ---
data_folder = "../data"
files_to_analyze = [
    "The Project Gutenberg eBook of Anna Karenina, by Leo Tolstoy.txt",
    "The Project Gutenberg eBook of War and Peace, by Leo Tolstoy.txt"
]

results = {} 

print(f"{'Book Name':<60} | {'Density':<10} | {'Avg Sen Len'}")
print("-" * 90)

for filename in files_to_analyze:
    file_path = os.path.join(data_folder, filename)
    
    # Check if file exists (handle both relative path and current dir)
    if not os.path.exists(file_path):
        file_path = filename 
        
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
            density, sen_lengths = calculate_metrics(text)
            
            # Clean up the name for the chart
            clean_name = filename.replace("The Project Gutenberg eBook of ", "").replace(".txt", "")
            
            # Store result for plotting
            results[clean_name] = {'density': density, 'lengths': sen_lengths}
            
            # Calculate avg length for print output
            avg_len = sum(sen_lengths) / len(sen_lengths) if sen_lengths else 0
            print(f"{clean_name:<60} | {density:.2f}%     | {avg_len:.1f} words")
            
    except FileNotFoundError:
        print(f"❌ Error: Could not find '{filename}'")

In [ ]:
# --- Visualization ---
if results:
    names = list(results.keys())
    
    # Create a layout with 2 Graphs (1 Row, 2 Columns)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Graph 1: Lexical Density (Bar Chart)
    densities = [results[n]['density'] for n in names]
    bars = ax1.bar(names, densities, color=['#4CAF50', '#2196F3'])
    ax1.set_title('Lexical Density Comparison')
    ax1.set_ylabel('Density (%)')
    ax1.set_ylim(0, 60)
    
    # Add numbers on bars
    for bar in bars:
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height(), 
                 f'{bar.get_height():.2f}%', ha='center', va='bottom')

    # Graph 2: Sentence Length Distribution (Box Plot)
    lengths_data = [results[n]['lengths'] for n in names]
    
    # Filter outliers > 80 words for cleaner visualization
    filtered_data = [[l for l in d if l < 80] for d in lengths_data]
    
    ax2.boxplot(filtered_data, labels=names, patch_artist=True, 
                boxprops=dict(facecolor='#FFC107'))
    ax2.set_title('Sentence Length Distribution (Box Plot)')
    ax2.set_ylabel('Words per Sentence')
    
    plt.tight_layout()
    plt.show()